# Preprocessing

In [2]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion SNR

In [36]:
def calculate_snr_from_signal(audio_signal, frame_size=1024, threshold=0.1):
    """
    Berechnet das SNR (Signal-zu-Rausch-Verhältnis) aus einem einzigen Signal.

    Args:
        audio_signal (np.array): Das Audio-Signal (1D-Array).
        frame_size (int): Die Anzahl der Samples pro Frame zur Analyse.
        threshold (float): Energie-Schwellenwert, um Rauschabschnitte zu identifizieren.

    Returns:
        float: Das geschätzte SNR in Dezibel.
    """
    # Aufteilen des Signals in Frames
    num_frames = len(audio_signal) // frame_size
    frames = audio_signal[:num_frames * frame_size].reshape((num_frames, frame_size))

    # Energie jedes Frames berechnen
    frame_energies = np.sum(frames**2, axis=1)

    # Rausch- und Signalenergie trennen
    noise_frames = frame_energies[frame_energies < threshold]  # Frames mit niedriger Energie
    signal_frames = frame_energies[frame_energies >= threshold]  # Frames mit hoher Energie

    # Mittelwert der Signal- und Rauschenergie
    noise_energy = np.mean(noise_frames) if len(noise_frames) > 0 else 1e-10
    signal_energy = np.mean(signal_frames) if len(signal_frames) > 0 else 1e-10

    # SNR berechnen
    snr = 10 * np.log10(signal_energy / (noise_energy + 1e-10))
    return snr

In [ ]:
import numpy as np

def calculate_snr_from_signal(audio_signal, frame_size=1024, threshold=0.01):
    """
    Berechnet das SNR (Signal-zu-Rausch-Verhältnis) aus einem einzigen Signal.

    Args:
        audio_signal (np.array): Das Audio-Signal (1D-Array).
        frame_size (int): Die Anzahl der Samples pro Frame zur Analyse.
        threshold (float): Energie-Schwellenwert, um Rauschabschnitte zu identifizieren.

    Returns:
        float: Das geschätzte SNR in Dezibel.
    """
    # Aufteilen des Signals in Frames
    num_frames = len(audio_signal) // frame_size
    frames = audio_signal[:num_frames * frame_size].reshape((num_frames, frame_size))

    # Energie jedes Frames berechnen
    frame_energies = np.sum(frames**2, axis=1)

    # Rausch- und Signalenergie trennen
    noise_frames = frame_energies[frame_energies < threshold]  # Frames mit niedriger Energie
    signal_frames = frame_energies[frame_energies >= threshold]  # Frames mit hoher Energie

    # Mittelwert der Signal- und Rauschenergie
    noise_energy = np.mean(noise_frames) if len(noise_frames) > 0 else 1e-10
    signal_energy = np.mean(signal_frames) if len(signal_frames) > 0 else 1e-10

    # SNR berechnen
    snr = 10 * np.log10(signal_energy / noise_energy)
    return snr

# Test

In [38]:
import os

# Pfad zum Ordner mit Audiodateien
input_dir = "../audio_files"

# Test: Log Energy für jede Datei berechnen
for file_name in sorted(os.listdir(input_dir)):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join(input_dir, file_name)
        
        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)
        
        # Log Energy berechnen
        snr_value = calculate_snr_from_signal(audio_signal)
        
        print(f"SNR ({file_path}): {snr_value:.2f} dB")

SNR (../audio_files/_noise_pink.wav): 126.17 dB
SNR (../audio_files/_noise_white.wav): 120.66 dB
SNR (../audio_files/_signal_constant.wav): 127.09 dB
SNR (../audio_files/_signal_silence.wav): 0.00 dB
SNR (../audio_files/_signal_sine.wav): 124.08 dB
SNR (../audio_files/_snr_03.wav): 109.39 dB
SNR (../audio_files/_snr_10.wav): 109.07 dB
SNR (../audio_files/_snr_20.wav): 23.84 dB
SNR (../audio_files/example1.wav): 31.78 dB
SNR (../audio_files/example2.wav): 31.65 dB
SNR (../audio_files/example3.wav): 28.94 dB
